# 套件安裝

In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests

# 設定初始化

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] Concentric: Literary and Cultural Studies（同心圓：文學與文化研究）
http://www.concentric-literature.url.tw/
[2] sqlite3 --- SQLite 数据库 DB-API 2.0 接口模块
https://docs.python.org/zh-tw/3/library/sqlite3.html
'''


'''
匯入套件
'''
# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 取得錯誤訊息
import sys, traceback

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# SQLite 資料庫
import sqlite3

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 取得系統時間的工具
from datetime import datetime

# 引入 hashlib 模組
import hashlib

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 給 web driver 用的變數
driver = None

# 來源首頁
prefix_url = 'http://www.concentric-literature.url.tw'
url = prefix_url + ''

# 指定 sheet name
folderName = sheetName = 'www_concentric_literature_url_tw'

# 指定 json 檔名
jsonFileName = f'{folderName}.json'

# 建立儲存檔案用的資料夾
folderPath = f'./{folderName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": folderPath
})

# 放置爬取的資料
listData = []

# iframe 網址
listIframeLinks = []

# 自訂函式 (網路爬蟲執行流程)

In [ ]:
'''
函式
'''
# md5 (用來為每一筆資料建立唯一代號)
def md5(string):
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    return m.hexdigest()

# 初始化 Web Driver
def init():
    global driver
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = driver_exec_path
    )
    
# 走訪頁面
def visit():
    global driver
    
    try:
        # 走訪首頁
        driver.get(url)

        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div.th_year > div.year_block > div.md_year > div.year_rt')
            )
        )
        
    except TimeoutException as e:
        print('等待逾時: visit')
    
# 剖析元素資料
def parse():
    global driver
    
    try:
        # 過濾出 iframe 路徑
        regexIframeLinks = r'\/lig[nh]tbox\d{4}-\d{1}\.html'
        
        # 整理出所有 iframe 路徑
        div_year_elms = driver.find_elements(By.CSS_SELECTOR, 'div.th_year > div.year_block > div.md_year > div.year_rt')
        for div in div_year_elms:
            strInnerText = div.get_attribute('innerHTML')
            listIframeLinks.extend(re.findall(regexIframeLinks, strInnerText))
            
        # 更新成正確的 iframe 網址
        for index in range(len(listIframeLinks)):
            listIframeLinks[index] = prefix_url + listIframeLinks[index]
            
        # 過濾刊號與出版日期
        regexPublish = r'(.+)\s\|\s(.+)'
            
        # 走訪所有 iframe 內頁
        for link in listIframeLinks:
            # 等待
            sleep(randint(1,2))
            
            # 走訪內頁
            driver.get(link)
            
            # 等待目標元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'div#wrap > main div.single-item div.slider-item')
                )
            )
            
            # 取得期刊標題
            strJournalTitle = driver.find_elements(By.CSS_SELECTOR, 'div.single-item div.slider-item header h4')[0].get_attribute('innerText').strip()
            
            # 取得刊號與出版日期
            strPublish = driver.find_elements(By.CSS_SELECTOR, 'div.single-item div.slider-item header span.booktime')[0].get_attribute('innerText').strip()
            matchPublish = re.search(regexPublish, strPublish)
            strPublishNum = matchPublish[1]
            strPublishDate = matchPublish[2]
            
            # 取得主要元素
            a_elms = driver.find_elements(By.CSS_SELECTOR, 'div.single-item div.slider-item:not([class~=slick-cloned]) div.th_lightbox a.md_book[href$=pdf]')
            for a in a_elms:
                # 篇名(書名)
                strJournalName = a.find_element(By.CSS_SELECTOR, 'div > p').get_attribute('innerText').strip()
                
                # 作者
                strAuthor = a.find_element(By.CSS_SELECTOR, 'header').get_attribute('innerText').strip()
                
                # pdf 連結
                strPdfLink = a.get_attribute('href')
                
                # 整理資料
                listData.append({
                    'id': md5(strPdfLink),
                    '刊名': strJournalTitle,
                    '刊號': strPublishNum,
                    '出版日期': strPublishDate,
                    '篇名': strJournalName,
                    '作者': strAuthor,
                    '網頁': link,
                    'pdf連結': strPdfLink
                })
        
    except TimeoutException as e:
        print('等待逾時: parse')
    
        
# 關閉瀏覽器
def close():
    global driver
    driver.quit()
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )
    
# 儲存 .db
def saveDB():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:      
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料，看看是否已經存在於資料表當中
        sql_query = f'''
        SELECT 1
        FROM journals
        WHERE id = ?
        '''
        
        # 寫入資料
        sql_insert = f'''
        INSERT INTO journals (
            id, journal_title, publish_num, publish_date, journal_sub_title, 
            author, link, pdf_link, is_downloaded, created_at, 
            updated_at
        ) VALUES ( 
            ?,?,?,?,?,
            ?,?,?,?,?,
            ?
        )
        '''
        
        # 放置準備寫入的資料
        list_insert = []
        
        # 將 json 資料一筆一筆找出來
        for myDict in listJson:
            # 如果資料庫沒有這筆資料(透過 id)，則將資料以 tuple 格式放到 list 當中，方便新增 bulk 資料
            if cursor.execute(sql_query, (myDict["id"],)).fetchone() == None:
                # 整合所有需要寫入的資料
                list_insert.append((
                    myDict['id'],
                    myDict['刊名'],
                    myDict['刊號'],
                    myDict['出版日期'],
                    myDict['篇名'],
                    myDict['作者'],
                    myDict['網頁'],
                    myDict['pdf連結'],
                    0,
                    datetime.today().strftime("%Y-%m-%d %H-%M-%S"),
                    datetime.today().strftime("%Y-%m-%d %H-%M-%S")
                ))
        
        # 新增資料到資料庫當中
        cursor.executemany(sql_insert, list_insert)
        
        # 執行 SQL 語法
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()
    
# 下載
def download():
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 將查詢出來的結果 (tuple)，變成 key-value 型式 (dict)
    conn.row_factory = sqlite3.Row
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢尚未下載的資料
        sql_query = f'''
        SELECT sn, id, pdf_link
        FROM journals
        WHERE `is_downloaded` = 0
        '''
        
        # 更新資料的欄位(是否下載過)
        sql_update = f'''
        UPDATE `journals` 
        SET 
            `is_downloaded` = 1 ,
            `updated_at` = ?
        WHERE `id` = ?
        '''
            
        # 取得所有未下載的資料
        for myDict in cursor.execute(sql_query).fetchall():
            # 等待
            sleep(randint(1,2))

            # 下載 pdf
            cmd = [
                'curl', 
                '-H', 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
                '-L', myDict["pdf_link"], 
                '-o', f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf'
            ]
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            #output = result.stdout
            #pprint(output)
            print(f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf')

            # 將 is_downloaded 改成 1，代表已下載過
            cursor.execute(sql_update, (datetime.today().strftime("%Y-%m-%d %H-%M-%S"), myDict["id"],))
                
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()

# 以下函式，請各別依情況分別、陸續執行

In [ ]:
# 初始化 Web Driver
init()

In [ ]:
# 走訪頁面
visit()

In [ ]:
# 剖析元素資料
parse()

In [ ]:
# 關閉瀏覽器
close()

In [ ]:
# 儲存成 json
saveJson()

In [ ]:
# 儲存 .db
saveDB()

In [ ]:
# 下載
download()